<a href="https://colab.research.google.com/github/gouthamans14/Transformers_Notebooks/blob/main/NER_CustomDataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers

In [ ]:
! pip install accelerate -U

In [155]:
import nltk
from nltk.corpus import brown

In [156]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [157]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [158]:
corpus = brown.tagged_sents(tagset="universal")

In [159]:
corpus

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [160]:
inputs = []
targets = []

for sentence_tag_pairs in corpus:
    tokens = []
    target = []
    for token, tag in sentence_tag_pairs:
        tokens.append(token)
        target.append(tag)
    inputs.append(tokens)
    targets.append(target)




In [161]:
inputs[0][:10]

['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of']

In [162]:
targets[0][:10]

['DET', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'DET', 'NOUN', 'ADP']

In [163]:
import json

In [164]:
with open("data1.json", 'w') as f:
    for x, y in zip(inputs,targets):
        j = {'inputs':x, 'targets':y}
        data = json.dumps(j)
        f.write(f"{data}\n")

In [165]:
data

'{"inputs": ["From", "what", "I", "was", "able", "to", "gauge", "in", "a", "swift", ",", "greedy", "glance", ",", "the", "figure", "inside", "the", "coral-colored", "boucle", "dress", "was", "stupefying", "."], "targets": ["ADP", "DET", "PRON", "VERB", "ADJ", "ADP", "NOUN", "ADP", "DET", "ADJ", ".", "ADJ", "NOUN", ".", "DET", "NOUN", "ADP", "DET", "ADJ", "NOUN", "NOUN", "VERB", "VERB", "."]}'

In [166]:

! pip install transformers

In [167]:
! pip install datasets

In [168]:
from datasets import load_dataset

In [169]:
data = load_dataset("json", data_files ='/content/data1.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [170]:
data

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 57340
    })
})

In [171]:
small_data = data['train'].shuffle(seed=42).select(range(20_000))

In [172]:
small_data

Dataset({
    features: ['inputs', 'targets'],
    num_rows: 20000
})

In [173]:
data = small_data.train_test_split(seed=42)

In [174]:
(data['train']['inputs'],
data['train']['targets'][1])

([['Ulyate',
   'and',
   'Kearton',
   'climbed',
   'on',
   'toward',
   'the',
   'sound',
   'of',
   'the',
   'barking',
   'of',
   'the',
   'dogs',
   'and',
   'the',
   'sporadic',
   'roaring',
   'of',
   'the',
   'lion',
   ',',
   'till',
   'they',
   'came',
   ',',
   'out',
   'of',
   'breath',
   ',',
   'to',
   'the',
   'crest',
   ',',
   'and',
   'peering',
   'through',
   'the',
   'branches',
   'of',
   'a',
   'bush',
   ',',
   'this',
   'is',
   'what',
   'Ulyate',
   'saw',
   ':',
   'Jones',
   'who',
   'had',
   'apparently',
   '(',
   'and',
   'actually',
   'had',
   ')',
   'ridden',
   'up',
   'the',
   'nearly',
   'impassable',
   'hillside',
   ',',
   'sitting',
   'calmly',
   'on',
   'his',
   'horse',
   'within',
   'forty',
   'feet',
   'of',
   'a',
   'full-grown',
   'young',
   'lioness',
   ',',
   'who',
   'was',
   'crouched',
   'on',
   'a',
   'flat',
   'rock',
   'and',
   'seemed',
   'just',
   'about',
   'to'

In [175]:
target_set = set()
for target in targets:
    target_set = target_set.union(target)
target_set

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [176]:
target_list = list(target_set)
id2label = {k:v for k,v in enumerate(target_list)}
label2id = {v:k for k,v in enumerate(target_list)}

In [177]:
target_list

['NUM',
 'VERB',
 '.',
 'DET',
 'ADJ',
 'ADP',
 'ADV',
 'PRT',
 'PRON',
 'X',
 'CONJ',
 'NOUN']

In [178]:
id2label

{0: 'NUM',
 1: 'VERB',
 2: '.',
 3: 'DET',
 4: 'ADJ',
 5: 'ADP',
 6: 'ADV',
 7: 'PRT',
 8: 'PRON',
 9: 'X',
 10: 'CONJ',
 11: 'NOUN'}

In [179]:
label2id


{'NUM': 0,
 'VERB': 1,
 '.': 2,
 'DET': 3,
 'ADJ': 4,
 'ADP': 5,
 'ADV': 6,
 'PRT': 7,
 'PRON': 8,
 'X': 9,
 'CONJ': 10,
 'NOUN': 11}

### Creating Auto Tokenizer

In [180]:
from transformers import AutoTokenizer

In [181]:
checkpoint ="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [183]:
data

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 5000
    })
})

In [184]:
t = tokenizer(data["train"][0]['inputs'],is_split_into_words=True)

In [185]:
t

{'input_ids': [101, 17359, 26139, 2063, 1998, 17710, 8445, 2239, 6589, 2006, 2646, 1996, 2614, 1997, 1996, 19372, 1997, 1996, 6077, 1998, 1996, 24590, 17197, 1997, 1996, 7006, 1010, 6229, 2027, 2234, 1010, 2041, 1997, 3052, 1010, 2000, 1996, 11146, 1010, 1998, 16740, 2083, 1996, 5628, 1997, 1037, 5747, 1010, 2023, 2003, 2054, 17359, 26139, 2063, 2387, 1024, 3557, 2040, 2018, 4593, 1006, 1998, 2941, 2018, 1007, 15230, 2039, 1996, 3053, 17727, 12054, 3085, 18573, 1010, 3564, 12885, 2006, 2010, 3586, 2306, 5659, 2519, 1997, 1037, 2440, 1011, 4961, 2402, 7006, 7971, 1010, 2040, 2001, 14275, 2006, 1037, 4257, 2600, 1998, 2790, 2074, 2055, 2000, 3715, 2032, 1010, 2096, 1996, 6077, 17097, 2105, 2014, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [186]:
t.word_ids()

[None,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 46,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 62,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 75,
 75,
 76,
 77,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 None]

In [187]:
t.tokens()

['[CLS]',
 'ul',
 '##yat',
 '##e',
 'and',
 'ke',
 '##art',
 '##on',
 'climbed',
 'on',
 'toward',
 'the',
 'sound',
 'of',
 'the',
 'barking',
 'of',
 'the',
 'dogs',
 'and',
 'the',
 'sporadic',
 'roaring',
 'of',
 'the',
 'lion',
 ',',
 'till',
 'they',
 'came',
 ',',
 'out',
 'of',
 'breath',
 ',',
 'to',
 'the',
 'crest',
 ',',
 'and',
 'peering',
 'through',
 'the',
 'branches',
 'of',
 'a',
 'bush',
 ',',
 'this',
 'is',
 'what',
 'ul',
 '##yat',
 '##e',
 'saw',
 ':',
 'jones',
 'who',
 'had',
 'apparently',
 '(',
 'and',
 'actually',
 'had',
 ')',
 'ridden',
 'up',
 'the',
 'nearly',
 'imp',
 '##ass',
 '##able',
 'hillside',
 ',',
 'sitting',
 'calmly',
 'on',
 'his',
 'horse',
 'within',
 'forty',
 'feet',
 'of',
 'a',
 'full',
 '-',
 'grown',
 'young',
 'lion',
 '##ess',
 ',',
 'who',
 'was',
 'crouched',
 'on',
 'a',
 'flat',
 'rock',
 'and',
 'seemed',
 'just',
 'about',
 'to',
 'charge',
 'him',
 ',',
 'while',
 'the',
 'dogs',
 'whirled',
 'around',
 'her',
 '.',
 '[SEP]'

In [188]:
label2id

{'NUM': 0,
 'VERB': 1,
 '.': 2,
 'DET': 3,
 'ADJ': 4,
 'ADP': 5,
 'ADV': 6,
 'PRT': 7,
 'PRON': 8,
 'X': 9,
 'CONJ': 10,
 'NOUN': 11}

In [189]:
label2id["NUM"]

0

In [190]:
def align_targets(labels, word_ids):
    aligned_labels=[]
    for word in word_ids:
        #print(word)
        if word is None:
            label=-100
        else:
            label = label2id[labels[word]]
        aligned_labels.append(label)
        #print(aligned_labels)

    return aligned_labels


In [191]:
data

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 5000
    })
})

In [192]:
labels = data['train']['targets']


In [193]:
t.tokens()[:10]

['[CLS]', 'ul', '##yat', '##e', 'and', 'ke', '##art', '##on', 'climbed', 'on']

In [194]:
t.word_ids()[:10]

[None, 0, 0, 0, 1, 2, 2, 2, 3, 4]

In [195]:
labels[0][t.word_ids()[8]]

'VERB'

### Check if aligned labels are working

In [196]:
data['train'][0]['targets']

['NOUN',
 'CONJ',
 'NOUN',
 'VERB',
 'PRT',
 'ADP',
 'DET',
 'NOUN',
 'ADP',
 'DET',
 'NOUN',
 'ADP',
 'DET',
 'NOUN',
 'CONJ',
 'DET',
 'ADJ',
 'NOUN',
 'ADP',
 'DET',
 'NOUN',
 '.',
 'ADP',
 'PRON',
 'VERB',
 '.',
 'PRT',
 'ADP',
 'NOUN',
 '.',
 'ADP',
 'DET',
 'NOUN',
 '.',
 'CONJ',
 'VERB',
 'ADP',
 'DET',
 'NOUN',
 'ADP',
 'DET',
 'NOUN',
 '.',
 'DET',
 'VERB',
 'DET',
 'NOUN',
 'VERB',
 '.',
 'NOUN',
 'PRON',
 'VERB',
 'ADV',
 '.',
 'CONJ',
 'ADV',
 'VERB',
 '.',
 'VERB',
 'ADP',
 'DET',
 'ADV',
 'ADJ',
 'NOUN',
 '.',
 'VERB',
 'ADV',
 'ADP',
 'DET',
 'NOUN',
 'ADP',
 'NUM',
 'NOUN',
 'ADP',
 'DET',
 'ADJ',
 'ADJ',
 'NOUN',
 '.',
 'PRON',
 'VERB',
 'VERB',
 'ADP',
 'DET',
 'ADJ',
 'NOUN',
 'CONJ',
 'VERB',
 'ADV',
 'ADV',
 'PRT',
 'VERB',
 'PRON',
 '.',
 'ADP',
 'DET',
 'NOUN',
 'VERB',
 'ADP',
 'PRON',
 '.']

In [197]:
idx =0
labels = data['train'][idx]['targets']
#print(labels[:10])
#print(word_ids[:30])
word_ids = t.word_ids()

#print(word_ids[:10])
aligned_targets = align_targets(labels, word_ids)


In [198]:
aligned_labels = [id2label[i] if i>=0 else None for i in aligned_targets]
for x, y in zip(t.tokens(), aligned_labels):
    print(f"{x}\t{y}")

[CLS]	None
ul	NOUN
##yat	NOUN
##e	NOUN
and	CONJ
ke	NOUN
##art	NOUN
##on	NOUN
climbed	VERB
on	PRT
toward	ADP
the	DET
sound	NOUN
of	ADP
the	DET
barking	NOUN
of	ADP
the	DET
dogs	NOUN
and	CONJ
the	DET
sporadic	ADJ
roaring	NOUN
of	ADP
the	DET
lion	NOUN
,	.
till	ADP
they	PRON
came	VERB
,	.
out	PRT
of	ADP
breath	NOUN
,	.
to	ADP
the	DET
crest	NOUN
,	.
and	CONJ
peering	VERB
through	ADP
the	DET
branches	NOUN
of	ADP
a	DET
bush	NOUN
,	.
this	DET
is	VERB
what	DET
ul	NOUN
##yat	NOUN
##e	NOUN
saw	VERB
:	.
jones	NOUN
who	PRON
had	VERB
apparently	ADV
(	.
and	CONJ
actually	ADV
had	VERB
)	.
ridden	VERB
up	ADP
the	DET
nearly	ADV
imp	ADJ
##ass	ADJ
##able	ADJ
hillside	NOUN
,	.
sitting	VERB
calmly	ADV
on	ADP
his	DET
horse	NOUN
within	ADP
forty	NUM
feet	NOUN
of	ADP
a	DET
full	ADJ
-	ADJ
grown	ADJ
young	ADJ
lion	NOUN
##ess	NOUN
,	.
who	PRON
was	VERB
crouched	VERB
on	ADP
a	DET
flat	ADJ
rock	NOUN
and	CONJ
seemed	VERB
just	ADV
about	ADV
to	PRT
charge	VERB
him	PRON
,	.
while	ADP
the	DET
dogs	NOUN
whirled	VERB
aroun

In [199]:
def tokenize_fn(batch):
    tokenized_inputs = tokenizer(batch['inputs'], truncation =True, is_split_into_words=True)

    labels_batch = batch['targets']
    aligned_labels_batch = []
    for i, labels in enumerate(labels_batch):
        word_ids = tokenized_inputs.word_ids(i)
        aligned_labels_batch.append(align_targets(labels,word_ids))

    tokenized_inputs['labels'] = aligned_labels_batch

    return tokenized_inputs

In [200]:
data['train'].column_names

['inputs', 'targets']

In [201]:
tokenized_datasets = data.map(
    tokenize_fn,
    batched=True,
    remove_columns=data["train"].column_names
)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [202]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

In [203]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

In [204]:
from transformers import DataCollatorForTokenClassification

In [205]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

### Need to Flatten the Target:

* In Name Entitiy Recoginition we have targets in IOB format.
* So we used Seq Eval method for computing metrics, which is made for IOB formats.
* But here, we have targets in IOB format so we are trying to get a flattened list.
* Simple way to evaluate our model is to flattened the prediction and target into single list and compare.
* Then use sci-kit learn to find accuracy score.

In [206]:
def flatten(list_of_list):
    flattened = [val for sublist in list_of_list for val in sublist]
    return flattened

In [207]:
from sklearn.metrics import f1_score, accuracy_score

In [208]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    preds = np.argmax(logits,axis=-1)

    labels_jagged= [[t for t in label if t!=-100]for label in labels]

    pred_jagged = [[p for p ,t in zip(ps,ts) if t!=-100] for ps, ts in zip(preds,labels)]

    labels_flat = flatten(labels_jagged)
    preds_flat = flatten(pred_jagged)
    print(labels_flat)
    print(preds_flat)

    acc = accuracy_score(labels_flat,preds_flat)
    f1 = f1_score(labels_flat,preds_flat, average='macro')

    return {
        "accuracy":acc,
        "f1": f1
    }


In [209]:
labels = [[-100,0,0,1,2,1,-100]]
logits = np.array([
    [[0.8,0.1,0.1],
     [0,0.6,0.4],
     [0.8,0.1,0.1],
     [0.0,0.3,0.7],
     [0.5,0.2,0.3],
     [0.9,0.1,0.0],
     [0.8,0.1,0.1]],

])

compute_metrics((logits,labels))

[0, 0, 1, 2, 1]
[1, 0, 2, 0, 0]


{'accuracy': 0.2, 'f1': 0.13333333333333333}

### Modelling

In [210]:
from transformers import AutoModelForTokenClassification

In [211]:
model = AutoModelForTokenClassification.from_pretrained(checkpoint,id2label=id2label,
                                                        label2id=label2id)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [212]:
from transformers import TrainingArguments

In [213]:
training_args = TrainingArguments(
    "distelbert-finetuned-ner",
    evaluation_strategy ="epoch",
    num_train_epochs =2,
    save_strategy = "epoch",
)

In [214]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    data_collator = data_collator,

    )

In [215]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.060600,0.053601
2,0.025000,0.047775


TrainOutput(global_step=3750, training_loss=0.06476986516316732, metrics={'train_runtime': 436.437, 'train_samples_per_second': 68.738, 'train_steps_per_second': 8.592, 'total_flos': 752900764156992.0, 'train_loss': 0.06476986516316732, 'epoch': 2.0})

### Saving the Model

In [222]:
trainer.save_model("my_model")

In [218]:
from transformers import pipeline

In [225]:
ner = pipeline(
    "token-classification",
    model = "my_model",
    aggregation_strategy = "simple",
    device =0
)

OSError: ignored

In [ ]:
model(["Gouthaman is an AI engineer"])

In [ ]:
model.

In [ ]:
input_text = "John Doe is a software engineer."

# Tokenize input text
tokens = tokenizer(input_text, return_tensors="pt")

# Perform prediction
with torch.no_grad():
    outputs = model(**tokens)


In [ ]:
import torch